In [ ]:
"""
I completely dug this out of the main file not very intelligently, so at the moment this is likely messed up.
"""
%pip install -U "datasets>=2.14.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.


In [ ]:
# ──────────────────────────────────────────────────────────────
# Imports
# ──────────────────────────────────────────────────────────────
import json
import os
import re
import shutil
import shlex
import subprocess
from pathlib import Path
from glob import glob
from types import SimpleNamespace as ns
from collections import Counter

import polars as pl
import torch
from datasets import load_dataset, load_from_disk
from huggingface_hub import notebook_login
from google.colab import drive
from transformers import AutoTokenizer


# Setup

In [ ]:
# ──────────────────────────────────────────────────────────────
# Global variables and namespace for the run
# ──────────────────────────────────────────────────────────────
# Datagen script settings
datagen_args = {
    'name': 'demo3s_cot',
    'length': 12,
    'dimension': 9,
    'true_instance_rate': .5,
    'train_samples': 5_000,
    'test_samples': 200,
    'cot_rate': 1,
    'no_filler_rate': 0,
    'corruption_rate': 1.33
}

# Run namespace
presets = {
    'cot': ns(
        runtype='cot',
        save_tag='base',
        filler_tok=None
    ),
    'direct': ns(
        runtype='direct',
        save_tag='base',
        filler_tok=None
    ),
    'punct_space-dot': ns(
        runtype='punct',
        save_tag='space-dot',
        filler_tok=' .'
    ),
    'punct_dot': ns(
        runtype='punct',
        save_tag='dot',
        filler_tok='.'
    ),
    'punct_space-comma': ns(
        runtype='punct',
        save_tag='space-comma',
        filler_tok=' ,'
    ),
    'filler_ctrl14': ns(
        runtype='filler',
        save_tag='ctrl14',
        filler_tok=b'\x14'.decode('latin-1')
    ),
    'filler_space-neu': ns(
        runtype='filler',
        save_tag='space-neu',
        filler_tok=' neu'
    ),
    'filler_space-big': ns(
        runtype='filler',
        save_tag='space-big',
        filler_tok=' big'
    ),
    'names_Peter': ns(
        runtype='names',
        save_tag='Peter',
        filler_tok='Peter'
    ),
    'names_Albert': ns(
        runtype='names',
        save_tag='Albert',
        filler_tok='Albert'
    ),
    'names_Anna': ns(
        runtype='names',
        save_tag='Anna',
        filler_tok='Anna'
    ),
    'chemistry_space-ion': ns(
        runtype='chemistry',
        save_tag='space-ion',
        filler_tok=' ion'
    ),
    'chemistry_space-alk': ns(
        runtype='chemistry',
        save_tag='space-alk',
        filler_tok=' alk'
    ),
    'chemistry_space-electron': ns(
        runtype='chemistry',
        save_tag='space-electron',
        filler_tok=' electron'
    ),
    'math_space-pi': ns(
        runtype='math',
        save_tag='space-pi',
        filler_tok=' pi'
    ),
    'math_space-sqrt': ns(
        runtype='math',
        save_tag='space-sqrt',
        filler_tok=' sqrt'
    ),
    'math_space-cos': ns(
        runtype='math',
        save_tag='space-cos',
        filler_tok=' cos'
    ),
    'drawing_space-ink': ns(
        runtype='drawing',
        save_tag='space-ink',
        filler_tok=' ink'
    ),
    'drawing_space-brush': ns(
        runtype='drawing',
        save_tag='space-brush',
        filler_tok=' brush'
    ),
    'drawing_space-paint': ns(
        runtype='drawing',
        save_tag='space-paint',
        filler_tok=' paint'
    ),
}

# Core paths
BASE_DIR = "/content/drive/MyDrive/Colab_Files/repurposed_tokens"
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODEL_ID = "meta-llama/Llama-3.2-1B"  # or 3-8B
MODEL_NAME = MODEL_ID.split('/')[-1]

RAW_TRAIN = os.path.join(DATA_DIR, 'base_train_w_cot.csv')
RAW_TEST = os.path.join(DATA_DIR, 'base_test_w_cot.csv')
RAW_ARGS = os.path.join(DATA_DIR, 'args_demo3s_cot.json')
print("RAW_TRAIN: ", RAW_TRAIN, "\n")

# Set data paths for each
for cfg in presets.values():
    tag = f"/{cfg.save_tag}" if cfg.save_tag else ""
    cfg.data_dir = f"{BASE_DIR}/data/{cfg.runtype}{tag}"
    print(cfg.runtype, cfg.data_dir)

    cfg.train_tok = f"{cfg.data_dir}/train_tokenized"
    cfg.test_tok = f"{cfg.data_dir}/test_tokenized"


RAW_TRAIN:  /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/base_train_w_cot.csv 

cot /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/cot/base
direct /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/direct/base
punct /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/punct/space-dot
punct /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/punct/dot
punct /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/punct/space-comma
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/ctrl14
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/space-neu
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/space-big
names /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/names/Peter
names /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/names/Albert
names /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/names/Anna
chemistry /content/drive/MyDrive/Colab_Files/repurpo

In [ ]:
# ──────────────────────────────────────────────────────────────
# Github and Drive setup
# ──────────────────────────────────────────────────────────────
%cd /content

# Mount drive for data
drive.flush_and_unmount()
if os.path.exists('/content/drive') and not os.path.islink('/content/drive'):
    shutil.rmtree('/content/drive')
drive.mount('/content/drive', force_remount=True)

    # Now we can make the directories:
for cfg in presets.values():
    os.makedirs(cfg.data_dir, exist_ok=True)

# Filepaths
FILLER_DIR = "/content/fillerTokens"
ENTROPIX_DIR = "/content/entropix"

# Clone and setup repos
if not os.path.exists(FILLER_DIR):
    print("Cloning filler")
    !git clone https://github.com/BreckEmert/fillerTokens.git
    # !git clone https://github.com/JacobPfau/fillerTokens.git
    !pip install -r {FILLER_DIR}/requirements.txt

if not os.path.exists(ENTROPIX_DIR):
    print("Cloning entropix")
    !git clone https://github.com/BreckEmert/entropix.git

# Set HF cache
os.environ["HF_DATASETS_CACHE"] = "/content/hf_ds_cache"


/content
Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Cloning filler
Cloning into 'fillerTokens'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 92 (delta 31), reused 28 (delta 18), pack-reused 39 (from 1)
Receiving objects: 100% (92/92), 86.88 KiB | 4.57 MiB/s, done.
Resolving deltas: 100% (39/39), done.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
Cloning entropix
Cloning into 'entropix'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 449 (delta 158), reused 103 (delta 103), pack-reused 224 (from 1)
Receiving objects: 100% (449/449), 1.64 MiB | 11.25 MiB/s, done.
Resolving deltas: 100% (234/234), done.


In [ ]:
# ──────────────────────────────────────────────────────────────
# HF setup
# ──────────────────────────────────────────────────────────────
notebook_login()


# Math Dataset Generation Script

In [ ]:
# ──────────────────────────────────────────────────────────────
# Generate raw math data with scripts.data_match3
# ──────────────────────────────────────────────────────────────
# This must be run because the script is a module:
%cd {FILLER_DIR}

# CSVs already exist
if not(os.path.exists(RAW_TRAIN) and os.path.exists(RAW_TEST)):
# if True:
    # While you're figuring out filepathing this avoids accidental script runs
    # raise ValueError(f"Safety break.  Raw path: {RAW_TRAIN}")

    # Run the data generation script
    cmd = (
        f"python -m scripts.data_match3 "
        f"--name {datagen_args['name']} "
        f"--length {datagen_args['length']} "
        f"--dimension {datagen_args['dimension']} "
        f"--true_instance_rate {datagen_args['true_instance_rate']} "
        f"--train_samples {datagen_args['train_samples']} "
        f"--test_samples {datagen_args['test_samples']} "
        f"--cot_rate {datagen_args['cot_rate']} "
        f"--no_filler_rate {datagen_args['no_filler_rate']} "
        f"--corruption_rate {datagen_args['corruption_rate']} "
        f"--data_path {DATA_DIR}"
    )
    print("Generating ", RAW_TRAIN, cmd)
    subprocess.run(shlex.split(cmd))
    # If you need debug:
    # result = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    # print(result.stdout)
    # print(result.stderr)

    # Args:
        # --length 12  # number of tuples per instance
        # --dimension 3  # number of integers per tuple
        # --cot_rate 0.5  # fraction to annotate with CoT traces
        # --no_filler_rate 0  # 0 so every non-CoT is punctuation-filler
        #  --corruption_rate 1.33  # Generated examples have on average 1-2 digits flipped with 1.33, and have a higher chance of being wrong.  Global true rate is maintained, to be clear.  But if the example is off by only one then it's harder to spot why it's wrong.  So higher corruption_rate = harder False examples.
    # Generates:
        # args_demo3s_YYYY-MM-DD.json  # metadata
        # demo3s_trainset_YYYY-MM-DD.csv
        # demo3s_testset_YYYY-MM-DD.csv

    # Find the generated file
    new_train = glob(f"{DATA_DIR}/demo3s_cot_trainset_*.csv")[0]
    new_test = glob(f"{DATA_DIR}/demo3s_cot_testset_*.csv")[0]
    new_arg = glob(f"{DATA_DIR}/args_demo3s_cot_*.json")[0]

    # Remove the date from their names
    Path(new_train).rename(RAW_TRAIN)
    Path(new_test).rename(RAW_TEST)
    Path(new_arg).rename(new_arg[:-16] + ".json")

# Make sure the final csvs look good:
print(f"Train csv: {pl.scan_csv(RAW_TRAIN).describe()}\n")
print(f"Test csv: {pl.scan_csv(RAW_TEST).describe()}\n")
print(f"Config json: {json.load(open(RAW_ARGS))}")

!ls {cfg.data_dir}


/content/fillerTokens
Generating  /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/base_train_w_cot.csv python -m scripts.data_match3 --name demo3s_cot --length 12 --dimension 9 --true_instance_rate 0.5 --train_samples 5000 --test_samples 200 --cot_rate 1 --no_filler_rate 0 --corruption_rate 1.33 --data_path /content/drive/MyDrive/Colab_Files/repurposed_tokens/data
Train csv: shape: (9, 2)
┌────────────┬─────────────────────────────────┐
│ statistic  ┆  344400139 722176591 806166964… │
│ ---        ┆ ---                             │
│ str        ┆ str                             │
╞════════════╪═════════════════════════════════╡
│ count      ┆ 4999                            │
│ null_count ┆ 0                               │
│ mean       ┆ null                            │
│ std        ┆ null                            │
│ min        ┆  000034906 056047730 489292704… │
│ 25%        ┆ null                            │
│ 50%        ┆ null                            │
│ 75%     

In [ ]:
def label_stats(csv_path: str):
    """Returns % of True labels"""
    with open(csv_path) as f:
        cnt = Counter(line.rstrip().rsplit(" ", 1)[-1] for line in f)
    total = sum(cnt.values())
    return cnt, cnt.get("True", 0) / total

# Make sure all are near 50 (stochastic so won't be exact)
for split, csv_path in {
    "train": RAW_TRAIN,
    "test":  RAW_TEST,
}.items():
    counts, pct_true = label_stats(csv_path)
    print(f"{cfg_name} {split}: {counts} -> {pct_true:.2%} True")


NameError: name 'cfg_name' is not defined

In [ ]:
# AI GENERATED CELL
# OUTPUT SHOULD BE 0 bad rows for each
# sanity-check every row in RAW_TRAIN / RAW_TEST
import sys, json, polars as pl, numpy as np, os
sys.path.append(FILLER_DIR)               # repo root → enables src.*
from src.match3 import Match3             # ← correct import

cfg  = json.load(open(RAW_ARGS))
DIM  = cfg["dimension"]
MOD  = cfg["mod"]

def parse_matrix(prompt: str) -> np.ndarray:
    tup_strs = prompt.split(" P ")[0].strip().split()
    return np.asarray([[int(ch) for ch in tup] for tup in tup_strs], dtype=int)

def validate(csv_path: str) -> int:
    df = pl.read_csv(csv_path, has_header=False, new_columns=["prompt"])
    bad = 0
    for p in df["prompt"]:
        mat   = parse_matrix(p)
        truth = Match3(DIM, MOD, len(mat)).solve(mat)
        label = p.rsplit(" A ", 1)[-1].strip()
        if str(truth) != label:
            bad += 1
            if bad <= 3:                                   # show first few
                print("mismatch →", p[:120] + "…", "||", truth)
    print(os.path.basename(csv_path), "bad rows:", bad, "/", len(df))
    return bad

validate(RAW_TRAIN)
validate(RAW_TEST)


base_train_w_cot.csv bad rows: 0 / 5000
base_test_w_cot.csv bad rows: 0 / 200


0

# Make Tokenized Datasets

In [ ]:
# ──────────────────────────────────────────────────────────────
# Instantiate the pretrained tokenizer
# ──────────────────────────────────────────────────────────────
# Instantiate tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tok.pad_token = tok.eos_token

# Establish tokens we need to reference
tok.cot_begin_id = tok(" P", add_special_tokens=False)["input_ids"]
tok.cot_end_id = tok(" A", add_special_tokens=False)["input_ids"]
tok.true_id = tok(" True", add_special_tokens=False)["input_ids"]
tok.false_id = tok(" False", add_special_tokens=False)["input_ids"]
tok.space_id = tok(" ", add_special_tokens=False)["input_ids"][0]
tok.minus_id  = tok('-', add_special_tokens=False)['input_ids'][0]

assert all(len(id) == 1 for id in [
    tok.cot_begin_id, tok.cot_end_id, tok.true_id, tok.false_id
]), "Either cot_id or truthy_id is not ONE id"

tok.cot_begin_id = tok.cot_begin_id[0]
tok.cot_end_id = tok.cot_end_id[0]
tok.true_id = tok.true_id[0]
tok.false_id = tok.false_id[0]

# Set each preset's filler token
for cfg in presets.values():
    if cfg.filler_tok:
        cfg.filler_id = tok(
            cfg.filler_tok, add_special_tokens=False
        )["input_ids"]
        assert len(cfg.filler_id) == 1, f"tok({cfg.filler_tok}) is not one id."
        cfg.filler_id = cfg.filler_id[0]
    else:
        cfg.filler_id = None

# Save the tokenizer
tok.save_pretrained(os.path.join(BASE_DIR, 'tokenizer'))


('/content/drive/MyDrive/Colab_Files/repurposed_tokens/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/Colab_Files/repurposed_tokens/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/Colab_Files/repurposed_tokens/tokenizer/tokenizer.json')

In [ ]:
# ──────────────────────────────────────────────────────────────
# Generate Tokenized datasets
# ──────────────────────────────────────────────────────────────
# Function to tokenize and mask a batch
def tokenize_and_mask(batch):
    # We want the labels to be only on the answer for all runs
    ids_batch = tok(batch['prompt'], add_special_tokens=True)["input_ids"]
    labels = [[-100] * (len(ids)-1) + [ids[-1]] for ids in ids_batch]
    return {"input_ids": ids_batch, 'labels': labels}

# Read the raw csv (raw cot script generation) as a HF dataset
raw_ds = load_dataset(
    "csv",
    data_files={"train": RAW_TRAIN, "test": RAW_TEST},
    column_names=["prompt"]
)

# Tokenize the raw dataset
def encode(batch):
    enc = tok(batch["prompt"], add_special_tokens=False)
    ids = [[tok.bos_token_id] + seq for seq in enc["input_ids"]]
    return {
        "input_ids": ids,
        # only the final token (answer / eos) is supervised
        "labels": [[-100] * (len(x) - 1) + [x[-1]] for x in ids],
    }
tok_ds = raw_ds.map(encode, batched=True, remove_columns=["prompt"])


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# ──────────────────────────────────────────────────────────────
# Generate run-specific datasets
# ──────────────────────────────────────────────────────────────
# Function to replace the filler of runtype='filler' datasets
def replace_filler(ids: list[int], filler_id: int) -> list:
    cot_idx = ids.index(tok.cot_begin_id) + 1
    if filler_id:
        n_pairs = sum(t == tok.minus_id for t in ids)
        new_filler = [filler_id]*n_pairs + [tok.cot_end_id]
    else:
        # Then this is direct
        new_filler = []

    # Return the original q/a with the new filler
    return ids[:cot_idx] + new_filler + [ids[-1]]

# Loop through all presets
for name, cfg in presets.items():
    # Don't generate if it already exists
    if os.path.exists(cfg.train_tok) and os.path.exists(cfg.test_tok):
        print(f"{name} already exists.")
        continue

    print(f"Running {name}")
    if cfg.runtype == 'cot':
        # Base csv is just the cot generation
        final_ds = tok_ds
    else:
        # Need to replace the filler
        final_ds = tok_ds.map(
            lambda batch, fid=cfg.filler_id: {
                "input_ids": (new_ids := [
                    replace_filler(ids, fid) for ids in batch["input_ids"]
                ]),
                "labels": [[-100]*(len(ids)-1) + [ids[-1]] for ids in new_ids],
            }, batched=True
        )

    # Save to disk
    final_ds['train'].save_to_disk(cfg.train_tok)
    final_ds['test'].save_to_disk(cfg.test_tok)


Running cot


Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

Running direct


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

...

Running drawing_space-paint


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/200 [00:00<?, ? examples/s]

# Debug Cells

In [ ]:
# AI GENERATED DEBUG CELL
# ──────────────────────────────────────────────────────────────
# Super-verbose debug / inspection cell
# ──────────────────────────────────────────────────────────────
from datasets import load_from_disk
from itertools import islice

def inspect(ds, split_name, cfg_name):
    ids = ds[2]["input_ids"]
    lbl = ds[2]["labels"]
    toks = tok.convert_ids_to_tokens(ids, skip_special_tokens=False)

    print(f"\n── {cfg_name} | {split_name} | example {0} ──")
    print("pos | token                    | id         | label_id")
    print("----+---------------------------+------------+----------")
    for pos, (t, i, l) in enumerate(zip(toks, ids, lbl)):
        print(f"{pos:3d} | {t:<25.25} | {i:<10d} | {l}")
        if l != -100 and l != i:
            raise ValueError(
                f"Label/input mismatch at pos {pos}: input={i}, label={l}"
            )

for name, cfg in presets.items():
    # if name != 'filler_space-big':
    #     continue

    print(f"\n================= {name.upper()} =================")
    train_ds = load_from_disk(cfg.train_tok)
    test_ds = load_from_disk(cfg.test_tok)

    inspect(train_ds, "train", name)
    inspect(test_ds, "test", name)

print("\n✓ Finished inspecting all presets.")


Streaming output truncated to the last 5000 lines.
375 | 7                         | 22         | -100
376 | Ġ                         | 220        | -100
377 | 11                        | 806        | -100
378 | -                         | 12         | -100
379 | Ġ                         | 220        | -100
380 | 1                         | 16         | -100
381 | Ġ                         | 220        | -100
382 | 11                        | 806        | -100
383 | -                         | 12         | -100
384 | Ġ                         | 220        | -100
385 | 6                         | 21         | -100
386 | ĠA                        | 362        | -100
387 | ĠTrue                     | 3082       | 3082

── cot | test | example 0 ──
pos | token                    | id         | label_id
----+---------------------------+------------+----------
  0 | <|begin_of_text|>         | 128000     | -100
  1 | Ġ                         | 220        | -100
  2 | 395                  